In [1]:
import pandas as pd
import os
from urllib.parse import urlparse
import random
import os
import boto3
import pystac_client
from dotenv import load_dotenv

## Setup Environment

In [2]:
# Get credentials from environment variables
load_dotenv()
ACCESS_KEY_ID = os.environ.get("ACCESS_KEY_ID")
SECRET_ACCESS_KEY = os.environ.get("SECRET_ACCESS_KEY")

In [3]:
class S3Connector:
    """A clean connector for S3-compatible storage services"""

    def __init__(self, endpoint_url, access_key_id,
                 secret_access_key, region_name='default'):
        """Initialize the S3Connector with connection parameters"""
        self.endpoint_url = endpoint_url
        self.access_key_id = access_key_id
        self.secret_access_key = secret_access_key
        self.region_name = region_name

        # Create session
        self.session = boto3.session.Session()

        # Initialize S3 resource
        self.s3 = self.session.resource(
            's3',
            endpoint_url=self.endpoint_url,
            aws_access_key_id=self.access_key_id,
            aws_secret_access_key=self.secret_access_key,
            region_name=self.region_name
        )

        # Initialize S3 client
        self.s3_client = self.session.client(
            's3',
            endpoint_url=self.endpoint_url,
            aws_access_key_id=self.access_key_id,
            aws_secret_access_key=self.secret_access_key,
            region_name=self.region_name
        )

    def get_s3_client(self):
        """Get the boto3 S3 client"""
        return self.s3_client

    def get_s3_resource(self):
        """Get the boto3 S3 resource"""
        return self.s3

    def get_bucket(self, bucket_name):
        """Get a specific bucket by name"""
        return self.s3.Bucket(bucket_name)

    def list_buckets(self):
        """List all available buckets"""
        response = self.s3_client.list_buckets()
        if 'Buckets' in response:
            return [bucket['Name'] for bucket in response['Buckets']]
        return []

## Utility Functions

The following function helps convert S3 URIs from the STAC catalog into S3 keys that can be used for direct access.

In [4]:
ACCESS_KEY_ID = os.environ.get("ACCESS_KEY_ID")
SECRET_ACCESS_KEY = os.environ.get("SECRET_ACCESS_KEY")
ENDPOINT_URL = 'https://eodata.dataspace.copernicus.eu'
ENDPOINT_STAC = "https://stac.dataspace.copernicus.eu/v1/"
BUCKET_NAME = "eodata"
catalog = pystac_client.Client.open(ENDPOINT_STAC)
connector = S3Connector(
    endpoint_url=ENDPOINT_URL,
    access_key_id=ACCESS_KEY_ID,
    secret_access_key=SECRET_ACCESS_KEY,
    region_name='default'
)
# Get S3 client and resource from the connector instance
s3 = connector.get_s3_resource()
s3_client = connector.get_s3_client()
buckets = connector.list_buckets()
print("Available buckets:", buckets)

Available buckets: ['DIAS', 'EODATA']


In [5]:
def prepare_paths(path_dir):

    df_input = pd.read_csv(f"{path_dir}/input.csv")
    df_output = pd.read_csv(f"{path_dir}/target.csv")

    df_input["path"] = df_input["Name"].apply(lambda x: os.path.join(path_dir, "input", os.path.basename(x).replace(".SAFE","")))
    df_output["path"] = df_output["Name"].apply(lambda x: os.path.join(path_dir, "target", os.path.basename(x).replace(".SAFE","")))
    df_input["S3Path"] = df_input["S3Path"].apply(lambda x: f"s3:/{x}" if not x.startswith("s3://") else x)
    df_output["S3Path"] = df_output["S3Path"].apply(lambda x: f"s3:/{x}" if not x.startswith("s3://") else x)
    return df_input, df_output

version = "V4"
TRAIN_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{version}/train/"
VAL_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{version}/val/"
TEST_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{version}/test/"
df_train_input, df_train_output =  prepare_paths(TRAIN_DIR)
df_val_input, df_val_output =  prepare_paths(VAL_DIR)
df_test_input, df_test_output =  prepare_paths(TEST_DIR)

In [7]:
from sklearn.utils import shuffle

In [8]:
# Shuffle and reset index for training set
df_train_input, df_train_output = shuffle(df_train_input, df_train_output, random_state=42)
df_train_input = df_train_input.reset_index(drop=True)
df_train_output = df_train_output.reset_index(drop=True)

# Shuffle and reset index for validation set
df_val_input, df_val_output = shuffle(df_val_input, df_val_output, random_state=42)
df_val_input = df_val_input.reset_index(drop=True)
df_val_output = df_val_output.reset_index(drop=True)

# Shuffle and reset index for test set
df_test_input, df_test_output = shuffle(df_test_input, df_test_output, random_state=42)
df_test_input = df_test_input.reset_index(drop=True)
df_test_output = df_test_output.reset_index(drop=True)


In [40]:
df_train_input[:70]['Name'][0]

'S2B_MSIL1C_20210401T101559_N0500_R065_T32TMR_20230519T181041.SAFE'

In [20]:
product_url = df_train_input["S3Path"][0]
product_url

's3://eodata/Sentinel-2/MSI/L1C_N0500/2018/01/02/S2A_MSIL1C_20180102T102421_N0500_R065_T32TMT_20230904T182253.SAFE'

In [21]:
# Replace .SAFE with .zarr
zarr_filename =  os.path.split(product_url)[1].replace('.SAFE', '.zarr')
# Join the current directory path with the new filename
zarr_path = os.path.join("./", zarr_filename)

In [22]:
from eopf.common.constants import OpeningMode
from eopf.common.file_utils import AnyPath

S3_CONFIG = {
    "key": ACCESS_KEY_ID, # EDIT WITH YOUR S3 KEY
    "secret": SECRET_ACCESS_KEY, # EDIT WITH YOUR S3 SECRET KEY
    "client_kwargs": { "endpoint_url": ENDPOINT_URL, "region_name": "default"} # EDIT WITH YOUR CLIENT_KWARGS
}
# Add this parameter if you want to overwrite the output of the conversion if it already exists
target_store_config = dict(mode=OpeningMode.CREATE_OVERWRITE)

In [23]:
from eopf.store.convert import convert

convert(AnyPath(product_url, **S3_CONFIG), zarr_path, target_store_kwargs=target_store_config)

INFO:eopf.store.convert:MultiThread Convert enabled
INFO:eopf.daskconfig.dask_context_manager:Initialising an ClusterType.LOCAL cluster with client conf : {} ,cluster config {'n_workers': 1} and dask config {}
INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:33613
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40213'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:41975', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:41975
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:48800
INFO:eopf.daskconfig.dask_context_ma

S2A_MSIL1C_20180102T102421_N0500_R065_T32TMT_20230904T182253
├── Attributes:
│   {'other_metadata': {'L0_ancillary_data_quality': 'null',
│                       'L0_ephemeris_data_quality': 'null',
│                       'NUC_table_ID': 2,
│                       'SWIR_rearrangement_flag': 'null',
│                       'UTM_zone_identification': 'S2A_OPER_MSI_L1C_TL_S2RP_20230904T182253_A013218_T32TMT_N05.00',
│                       'absolute_location_assessment_from_AOCS': 'null',
│                       'band_description': {'01': {'bandwidth': '20.0',
│                                                   'central_wavelength': 442.7,
│                                                   'onboard_compression_rate': '2.655',
│                                                   'onboard_integration_time': '7.4473767',
│                                                   'physical_gain': '4.11821352',
│                                                   'spectral_response_step': '1',
│                                                   'spectral_response_values': '0.001775742 '
│                                                                               '0.004073061 '
│                                                                               '0.003626143 '
│                                                                               '0.003515199 '
│                                                                               '0.005729163 '
│                                                                               '0.003780292 '
│                                                                               '0.002636732 '
│                                                                               '0.001262113 '
│                                                                               '0.001987583 '
│                                                                               '0.001368913 '
│                                                                               '0.001250444 '
│                                                                               '0.000463454 '
│                                                                               '0.000814293 '
│                                                                               '0.001376431 '
│                                                                               '0.001485086 '
│                                                                               '0.001823735 '
│                                                                               '0.001626817 '
│                                                                               '0.004392062 '
│                                                                               '0.029008099 '
│                                                                               '0.11874593 '
│                                                                               '0.32387506 '
│                                                                               '0.57281921 '
│                                                                               '0.71472749 '
│                                                                               '0.76196778 '
│                                                                               '0.78929702 '
│                                                                               '0.80862387 '
│                                                                               '0.81089382 '
│                                                                               '0.82419876 '
│                                                                               '0.85415811 '
│                                                                               '0.87079088 '
│                                                                               '0.88731097 '
│                                                                        